## Critical path showcase

__UNDER CONSTRUCTION__

This notebook provides examples of a simulation that takes a number of barges and a total amount to be moved. We show the critical path (as a pd.DataFrame and a gantt plot) for a number of different simulation (data) and methods.

For this example we work with the following methods to determine critical path:
* __DependenciesFromRecordedActivities__
* __DependenciesFromSimpy__

For this example we work with the following simulations to determine critical path:
* two barges move an amount from site 0 to site 1. When the barges done, another (third) vessel moves the entire amount from site 1 to site 2.
* five barges move an amount from site 0 to site 1. When the barges done,another (sixth) vessel moves the entire amount from site 1 to site 2.


#### 0. Import libraries

In [1]:
import simpy
import shapely

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

from openclsim.critical_path.dependencies_from_simpy_step import DependenciesFromSimpy, AlteredStepEnv
from openclsim.critical_path.dependencies_from_recorded_activities import DependenciesFromRecordedActivities


#### 1. Use a function definition to get some simulation data

In this notebook we can play around with the nr of barges and total amount to be moved. After calling this function with desired input parameters we can investigate the data of the simulations and inspect the gantt chart.

In [2]:
def demo_data(nr_barges, total_amount, env=None):
    """
    Run a simulation where <nr_barges> barges need to shift an amount of <total_amount>
    from site 1 to site 2 whereafter a larger vessel can come into action.

    Parameters
    ----------
    nr_barges : int
        Number of barges in the simulation.
    total_amount : int
        Total amount to be transported in the simulation.
    env : simpy.Environment or class that inherits from simpy.Environment
        Optional. If None, default to simpy Environment
    """
    Site = type(
        "Site",
        (
            core.Identifiable,
            core.Log,
            core.Locatable,
            core.HasContainer,
            core.HasResource,
        ),
        {},
    )
    TransportProcessingResource = type(
        "TransportProcessingResource",
        (
            core.Identifiable,
            core.Log,
            core.ContainerDependentMovable,
            core.Processor,
            core.HasResource,
        ),
        {},
    )

    simulation_start = 0
    if env is None:
        my_env = simpy.Environment(initial_time=simulation_start)
    else:
        my_env = env(initial_time=simulation_start)

    registry = {}

    location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)

    data_from_site = {
        "env": my_env,
        "name": "from_site",
        "geometry": location_from_site,
        "capacity": 4 * total_amount,
        "level": 4 * total_amount,
        "nr_resources": 1,
    }
    from_site = Site(**data_from_site)

    location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)
    data_to_site = {
        "env": my_env,
        "name": "to_site",
        "geometry": location_to_site,
        "capacity": 3*total_amount,
        "level": 0,
        "nr_resources": 4,
    }
    to_site = Site(**data_to_site)

    location_to_site2 = shapely.geometry.Point(4.35222222, 52.11428333)
    data_to_site2 = {
        "env": my_env,
        "name": "to_site2",
        "geometry": location_to_site2,
        "capacity": total_amount,
        "level": 0,
        "nr_resources": 4,
    }
    to_site2 = Site(**data_to_site2)

    vessels = {}

    for i in range(nr_barges):
        vessels[f"vessel{i}"] = TransportProcessingResource(
            env=my_env,
            name=f"barge_{i}",
            geometry=location_from_site,
            capacity=10,
            compute_v=lambda x: 10,
        )

    # vessel_last wait till whiletask done
    vessel_last = TransportProcessingResource(
        env=my_env,
        name="vessel_last",
        geometry=location_from_site,
        capacity=10,
        compute_v=lambda x: 10,
    )
    vessels["vessel_last"] = vessel_last

    activities = {}
    for i in range(nr_barges):
        amount = 5  # handle loading
        duration = 2000  # sailing and unloading

        requested_resources = {}
        activities[f"activity{i}"] = model.WhileActivity(
            env=my_env,
            name=f"while_sequential_activity_subcycle{i}",
            registry=registry,
            sub_processes=[
                model.SequentialActivity(
                    env=my_env,
                    name=f"sequential_activity_subcycle{i}",
                    registry=registry,
                    sub_processes=[
                        model.BasicActivity(
                            env=my_env,
                            name="basic activity:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            duration=duration,
                            additional_logs=[vessels[f"vessel{i}"]],
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing empty:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            mover=vessels[f"vessel{i}"],
                            destination=from_site,
                            duration=duration - 200*i
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="loading:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            processor=vessels[f"vessel{i}"],
                            origin=from_site,
                            destination=vessels[f"vessel{i}"],
                            amount=amount,
                            duration=500 * amount,
                            requested_resources=requested_resources,
                            start_event=[
                            {
                                "type": "container",
                                "concept": from_site,
                                "state": "gt",
                                "level": amount
                            }
                            ]
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing full:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            mover=vessels[f"vessel{i}"],
                            destination=to_site,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="unloading:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            processor=vessels[f"vessel{i}"],
                            origin=vessels[f"vessel{i}"],
                            destination=to_site,
                            amount=amount,
                            duration=duration,
                            requested_resources=requested_resources,
                        ),
                    ],
                )
            ],
            condition_event=[
                {
                    "type": "container",
                    "concept": from_site,
                    "state": "lt",
                    "level": 2.8*total_amount
                }
            ],
              start_event={
                "type":"time",
                "start_time":i*100,
            }
        )

    # now add activity for vessel last, once v1 and v2 are done
    requested_resources = {}
    amount = 5
    duration = 2000
    activities[f"activity_last_vessel"] = model.WhileActivity(
            env=my_env,
            name=f"while_last_vessel",
            registry=registry,
            sub_processes=[
                model.SequentialActivity(
                    env=my_env,
                    name="sequential_last_vessel",
                    registry=registry,
                    sub_processes=[
                        model.BasicActivity(
                            env=my_env,
                            name="basic activity vessel_last",
                            registry=registry,
                            duration=duration,
                            additional_logs=[vessel_last]
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing empty: vessel_last",
                            registry=registry,
                            mover=vessel_last,
                            destination=to_site,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="loading vessel_last",
                            registry=registry,
                            processor=vessel_last,
                            origin=to_site,
                            destination=vessel_last,
                            amount=amount,
                            duration=duration,
                            requested_resources=requested_resources,
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing full vessel_last",
                            registry=registry,
                            mover=vessel_last,
                            destination=to_site2,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="unloading vessel_last",
                            registry=registry,
                            processor=vessel_last,
                            origin=vessel_last,
                            destination=to_site2,
                            amount=amount,
                            duration=duration,
                            requested_resources=requested_resources,
                        ),
                    ],
                )],
            start_event=[
                {
                    "type": "container",
                     "concept": to_site,
                     "state": "gt",
                     "level": 30
                }
            ],
            condition_event=[
                {
                    "type": "container",
                    "concept": to_site2,
                    "state": "full",
                }
            ],)

    model.register_processes(list(activities.values()))
    my_env.run()

    return {
        "env": my_env,
        "object_list": [from_site, to_site, to_site2] + list(vessels.values()),
        "activity_list": list(activities.values()),
    }



#### 3. Get critical path with simpy

* _method: DependenciesFromSimpy_
* _simulation: simulation with 2 barges_

First we run the simulation, then we get the recorded activities (with flag 'is_critical') and show these in a gantt plot.

In [3]:
simulation_data_out = demo_data(3, 100, env=AlteredStepEnv)


In [4]:
my_cp_dependencies_from_simpy = DependenciesFromSimpy(**simulation_data_out)
critical_df = my_cp_dependencies_from_simpy.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
0,d4449f17-eb8a-4bde-8cea-4e314c411a42,basic activity:barge_0,Activity,1970-01-01,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,411b187d-062d-4905-a973-b571e0bb22ba,False
1,d4449f17-eb8a-4bde-8cea-4e314c411a42,basic activity:barge_0,barge_0,1970-01-01,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,411b187d-062d-4905-a973-b571e0bb22ba,False
2,8cc5a9b1-1499-401c-a20f-0412a0b80e33,sequential_activity_subcycle0,Activity,1970-01-01,1970-01-01 01:06:40,0 days 01:06:40,ACTIVE,e12cc65f-0fe6-4336-81fa-4cfb4e6a5f3f,False
3,8cc5a9b1-1499-401c-a20f-0412a0b80e33,sequential_activity_subcycle0,Activity,1970-01-01,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,6230ea88-67ae-47c0-8c3d-db5f5c6a0726,False
4,2d2cc781-2167-4856-9e39-84340b1b4098,while_last_vessel,Activity,1970-01-01,1970-01-01 08:28:20,0 days 08:28:20,WAITING,22c22700-e8e2-4add-b455-eaff5fd41f47,False


In [5]:
plot.get_gantt_chart(simulation_data_out['object_list'],
                     critical_path_dataframe=my_cp_dependencies_from_simpy.get_plotly_gantt_data(),
                     id_map=simulation_data_out['activity_list'])
                     

In [8]:
my_cp_dependencies_from_simpy.get_plotly_plot()

ValueError: 
    Invalid value of type 'builtins.list' received for the 'name' property of scatter
        Received value: ['basic activity:barge_0', 'basic activity:barge_1', 'basic activity:barge_2', 'sailing empty:barge_0', 'sailing empty:barge_1', 'sailing empty:barge_2', 'loading:barge_2', 'loading:barge_1', 'sailing full:barge_2', 'unloading:barge_2', 'loading:barge_0', 'sailing full:barge_1', 'basic activity:barge_2', 'unloading:barge_1', 'sailing full:barge_0', 'sailing empty:barge_2', 'basic activity:barge_1', 'unloading:barge_0', 'loading:barge_2', 'sailing empty:barge_1', 'basic activity:barge_0', 'sailing full:barge_2', 'loading:barge_1', 'sailing empty:barge_0', 'unloading:barge_2', 'sailing full:barge_1', 'loading:barge_0', 'basic activity:barge_2', 'unloading:barge_1', 'sailing full:barge_0', 'sailing empty:barge_2', 'basic activity:barge_1', 'unloading:barge_0', 'loading:barge_2', 'sailing empty:barge_1', 'basic activity:barge_0', 'sailing full:barge_2', 'loading:barge_

#### 4. Get critical path with recorded activities

* _method: DependenciesFromRecordedActivities_
* _simulation: simulation with 2 barges_

First we run the simulation, then we get the recorded activities (with flag 'is_critical') and show these in a gantt plot.

In [6]:
# run simulation
simulation_data_out = demo_data(2, 100)

# get critical path as recorded_activities_dataframe with column 'is_critical'
my_cp_dependencies_from_activities = DependenciesFromRecordedActivities(**simulation_data_out)
critical_df = my_cp_dependencies_from_activities.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
0,d7eeb369-a533-4d91-b20a-4d04330bbfed,basic activity:barge_0,Activity,1970-01-01,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,77dcc8cd-ed9e-42e5-9197-1d1a90e79352,False
1,d7eeb369-a533-4d91-b20a-4d04330bbfed,basic activity:barge_0,barge_0,1970-01-01,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,77dcc8cd-ed9e-42e5-9197-1d1a90e79352,False
2,14768d7a-b8c4-4ab1-9f4a-5365bfd7d7d4,sequential_activity_subcycle0,Activity,1970-01-01,1970-01-01 01:06:40,0 days 01:06:40,ACTIVE,7fc6d176-bdf0-4ba0-96b8-5ca6b155af59,False
3,14768d7a-b8c4-4ab1-9f4a-5365bfd7d7d4,sequential_activity_subcycle0,Activity,1970-01-01,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,d3f1ed3f-8208-4840-8c96-c1b646e5c33e,False
4,848c39e6-ebd2-4a56-ac76-d9e1c61f9286,while_last_vessel,Activity,1970-01-01,1970-01-01 11:28:20,0 days 11:28:20,WAITING,538af510-f956-467e-9a83-4abc48688766,True


In [7]:
plot.get_gantt_chart(simulation_data_out['object_list'],
                     critical_path_dataframe=critical_df,
                     id_map=simulation_data_out['activity_list'])

#### 5. Get critical path with simpy

* _method: DependenciesFromSimpy_
* _simulation: simulation with 5 barges_

First we run the simulation, then we get the recorded activities (with flag 'is_critical') and show these in a gantt plot.

In [8]:
# run simulation
simulation_data_out = demo_data(12, 100, env=AlteredStepEnv)

# get critical path as recorded_activities_dataframe with column 'is_critical'
my_cp_dependencies_from_simpy = DependenciesFromSimpy(**simulation_data_out)
critical_df = my_cp_dependencies_from_simpy.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
0,49715422-aba0-45c2-8d77-bb60e1cfa2bb,basic activity:barge_0,barge_0,1970-01-01,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,5557db9a-374f-4cce-91e0-7184a2fea5cc,False
1,dc903fa8-d69d-4dcc-aae8-fc769aa1d957,basic activity:barge_1,barge_1,1970-01-01,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,50e19c7f-1a9b-4eea-a21f-fe78b78ef855,False
2,079072cc-c30c-44d6-ab15-4d69c3e87254,basic activity:barge_10,barge_10,1970-01-01,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,d53ef3c4-74b6-4a68-bbc9-88aa8e9fc229,False
3,60d121a0-5560-4fed-9a51-7743a81f4b67,basic activity:barge_11,barge_11,1970-01-01,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,23d36aab-4a2f-4a1d-a963-96f8f8279579,True
4,b8fec844-a31c-428e-95d5-65374636c89d,basic activity:barge_2,barge_2,1970-01-01,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,871c5058-5eba-4672-822c-ac207a14a24b,False


In [9]:
plot.get_gantt_chart(simulation_data_out['object_list'],
                     critical_path_dataframe=critical_df,
                     id_map=simulation_data_out['activity_list'])